In [ ]:
# UTTERANCE TO PHONEME MAPPING 

For this assignment we explore the mapping of spoken utterances to their corresponding phonemes. Which is a crucial step in Automatic Speech Recognition (ASR) systems, where the goal is to convert audio signals into a sequence of phonemes, which can then be used to reconstruct the original text.

For this assignment I used the 


# Installs

In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 1.13.1+cu117 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# !pip install --upgrade setuptools wheel
!pip install setuptools==69.5.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 70.0.0
    Uninstalling setuptools-70.0.0:
      Successfully uninstalled setuptools-70.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 24.9.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 1.13.1+cu117 which is incompatible.



This may take a while

In [3]:
!pip install torchsummaryX==1.3.0
!pip install pandas==1.5.2
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2024.66.154055 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.5 which is incompatible.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 

# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

from torch.autograd import Variable
import torchaudio.transforms as T

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [5]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"angeizabayo","key":"6727c97821f5c0aae21046c561c0545d"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [6]:
# !kaggle competitions download -c 11-785-hw3p2-f24

In [7]:
# '''
# This will take a couple minutes, but you should see at least the following:
# 11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data
# '''
# !unzip -q 11-785-hw3p2-f24.zip
# !ls

# Google Drive

In [8]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

Given CMU ARPAbet dictionary which maps ARPABET phoneme to their corresponding single chatacter used in our model


In [9]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]



In [10]:
# You might want to play around with the mapping as a sanity check here

### Train Data

Given AudioDataset Class to handles the loading, processing, and batching of speech data for phoneme recognition model. Where loads MFCC features and phoneme transcripts, provides methods for accessing individual data samples, and implements collate_fn method for efficient batch processing with padding and potential transformations.


In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,directory,partition):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''
        self.mfcc_dir       =  os.path.join(directory, partition,'mfcc')
        self.transcript_dir = os.path.join(directory,partition,'transcript')

        self.mfcc_files          = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files    = sorted(os.listdir(self.transcript_dir))


        self.PHONEMES = PHONEMES

        assert len(self.mfcc_files) == len(self.transcript_files)

        self.mfccs= []
        self.transcripts  = []

        for file in self.mfcc_files:
            files = np.load(os.path.join(self.mfcc_dir, file))
            self.mfccs.append((files))
        for file in self.transcript_files:
            files = np.load(os.path.join(self.transcript_dir, file))
            transcripts = files[1:-1]
            self.transcripts.append(np.array([self.PHONEMES.index(i) for i in transcripts]))


        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfccs)

        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = torch.FloatTensor(self.mfccs[ind])
        transcript = torch.LongTensor(self.transcripts[ind])

        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [i[0] for i in batch]
        # batch of output phonemes
        batch_transcript = [i[1] for i in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = [len(i) for i in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True) # TODO
        lengths_transcript = [len(i) for i  in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

 AudioDatasetTest Class designed to manage the test dataset for our speech recognition model. Where it loads only the MFCC features from the specified directory, as the test set does not include labels. It includes methods for accessing test samples and a  collate_fn method to handle padding and potential data transformations during the batching process, crucial for consistent input to the model during inference.


In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
    def __init__(self,directory,partition):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        self.mfcc_dir  =  os.path.join(directory, partition,'mfcc')
        self.mfcc_files  = sorted(os.listdir(self.mfcc_dir)) #NotImplemented
        self.PHONEMES = PHONEMES

        self.mfccs = []

        for files in self.mfcc_files:
            file_path = np.load(os.path.join(self.mfcc_dir,files))
            self.mfccs.append(file_path)

        self.length = len(self.mfcc_files)

    def __len__(self):
        return self.length


    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.FloatTensor(self.mfccs[ind])

        return mfcc


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
          look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
          Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
          and lengths of labels.
        '''
        batch_mfcc = batch
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = [len(i) for i in batch_mfcc]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, actual length of features
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)


### Config - Hyperparameters

updated config dictionary for saving the hyperparameters used in Assignment

In [ ]:
# Feel free to add more items here
config = {
    "beam_width" : 2,
    "lr"         : 1e-3,
    "epochs"     : 50,
    "batch_size" : 64,
    'directory': '/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2',
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations


### Data loaders

In [14]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [ ]:
from torch.utils.data import DataLoader
train_data = AudioDataset(directory = config['directory'], partition='train-clean-100')
val_data = AudioDataset(directory= config['directory'], partition='dev-clean')
test_data = AudioDatasetTest(directory=config['directory'], partition='test-clean')

train_loader = DataLoader(
    dataset = train_data,
    num_workers=2,
    batch_size =config['batch_size'],
    pin_memory=True, shuffle=True,
    collate_fn= train_data.collate_fn
)
val_loader = DataLoader(
    dataset = val_data,
    num_workers=2,
    batch_size =config['batch_size'],
    pin_memory=True, 
    shuffle=True,
    collate_fn= val_data.collate_fn
)
test_loader = DataLoader(
    dataset = test_data,
    num_workers=1,
    batch_size =config['batch_size'],
    pin_memory=True,
    shuffle=False,
    collate_fn= test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [16]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1683, 28]) torch.Size([64, 221]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

I did not use this basic block as it was suggested so I jumped straight to pBLSTM Network

In [ ]:
# torch.cuda.empty_cache()

# class Network(nn.Module):

#     def __init__(self):

#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         # self.embedding = ?

#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size = __, hidden_size = 256, num_layers = 1)

#         self.classification = nn.Sequential(
#             #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
#         )


#         self.logSoftmax = #TODO: Apply a log softmax here. Which dimension would apply it on ?

#     def forward(self, x, lx):
#         #TODO
#         # The forward function takes 2 parameter inputs here. Why?
#         # Refer to the handout for hints
#         pass

## Initialize Basic Network
(If trying out the basic Network)

In [ ]:
# torch.cuda.empty_cache()

# model = Network().to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

 Pyramid BI-LSTM(pBLSTM) with PermuteBlock for transposing tensor dimensions and pBLSTM
 A custom Pyramidal BiLSTM module designed to process sequences by reducing their time resolution while doubling the feature dimension, facilitating deeper and more efficient sequence modeling.



In [20]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size=input_size*2, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)
        # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        
        x, x_lens = pad_packed_sequence(x_packed, batch_first=True) # TODO: Pad Packed Sequence

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        x, x_lens = self.trunc_reshape(x, x_lens)

        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)


        # TODO: Pass the sequence through bLSTM
        x_packed, _ = self.blstm(x_packed)

        # What do you return?

        return x_packed

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor

        batch_size, timesteps, features = x.shape

        # If there is an odd number of timesteps, exclude the last timestep
        if timesteps % 2 != 0:
            x = x[:, :-1, :]
            x_lens -= 1

        # Reshape x
        x = x.view(batch_size, timesteps // 2, features * 2)

        # Reduce lengths by half
        x_lens = x_lens // 2

        return x, x_lens

Introduced class LockedDropout to applies dropout to an entire timestep across the batch, preserving the same dropout mask for all time steps.

In [ ]:
class LockedDropout(nn.Module):
    def __init__(self, dropout=0.25):
        super().__init__()
        self.dropout = dropout

    def forward(self, x):
        if not self.training or self.dropout == 0:
            return x
        un_packed, x_lens = pad_packed_sequence(x, batch_first=True)
        
        # Create a dropout mask
        m = x.data.new(1, un_packed.size(1), un_packed.size(2)).bernoulli_(1 - self.dropout)
        mask = Variable(m, requires_grad=False) / (1 - self.dropout)
        mask = mask.expand_as(un_packed)
        
        # Apply the mask to the unpacked input
        out = mask * un_packed
        return pack_padded_sequence(out, lengths=x_lens, batch_first=True, enforce_sorted=False)


### Encoder

Encoder class Consists of embedding layer using convolutions for feature extraction, and multiple layers of pyramidal BiLSTMs (pBLSTMs) that gradually reduce sequence length while maintaining feature complexity, and employs LockedDropout for improved regularization during training.

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, input_size, encoder_hidden_size, dropout=0.25):
        super(Encoder, self).__init__()
        self.embedding = nn.Sequential(
            PermuteBlock(),
            nn.Conv1d(in_channels=input_size, out_channels=input_size, kernel_size=7, padding=3),
            nn.GELU(),
            PermuteBlock()
        ) #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        
        
        # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            # ...
            # ...
        self.BLSTMs = nn.LSTM(
            input_size=input_size,
            hidden_size=encoder_hidden_size,
            num_layers=3, batch_first=True,
            bidirectional=True,
            dropout=dropout
        )
        
        self.pBLSTMs = nn.Sequential(
            LockedDropout(dropout),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size),
            LockedDropout(dropout),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size)
        )
        
        self.dropout_layer = LockedDropout(dropout)
        

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        x = self.embedding(x)    #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        x_packed, (h_n, c_n) = self.BLSTMs(x_packed)
        x_packed = self.pBLSTMs(x_packed)
        # TODO: Pass Sequence through the pyramidal Bi-LSTM laye
        # TODO: Pad Packed Sequence
        encoder_outputs, encoder_lens = pad_packed_sequence(x_packed, batch_first=True)
        # Remember the number of output(s) each function returns
        return encoder_outputs, encoder_lens

### Decoder

The Decoder class transforms encoder outputs to phoneme predictions using a multi-layer perceptron (MLP) architecture. This MLP includes layers for batch normalization, GELU activations, and dropout for regularization. The final output is produced by a linear layer followed by a log softmax activation to generate a probability distribution over the possible phonemes, suitable for classification tasks.


In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41, decoder_dropout=0.2):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(2*embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2 * embed_size, 2 * embed_size),
            torch.nn.GELU(),
            torch.nn.Dropout(decoder_dropout),
            torch.nn.Linear(2 * embed_size, 2 * embed_size),
            torch.nn.GELU(),
            PermuteBlock(),
            torch.nn.BatchNorm1d(2*embed_size), 
            PermuteBlock(),
            torch.nn.Linear(2 * embed_size, embed_size),
            torch.nn.GELU(),
            torch.nn.Dropout(decoder_dropout),
            torch.nn.Linear(embed_size, output_size)
            )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        mlp_out = self.mlp(encoder_out)
        #TODO Think what should be the final output of the decoder for the classification
        return self.softmax(mlp_out)
        

 Automatic Speech Recognition (ASR) network, I included data augmentations such as time and frequency masking to enhance robustness against variations in input data. Additianally, initialized Encoder and Decoder aswell for input processing and produce the final phoneme sequence respectively.


In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            T.TimeMasking(time_mask_param=10),
            T.FrequencyMasking(freq_mask_param=5)
            #Hint: See how to use PermuteBlock() function defined above
        )
        self.encoder        = Encoder(input_size, embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size, output_size)# TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [28]:
model = ASRModel(
    input_size  = 28, #TODO,
    embed_size  = 256, #TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential(
    (0): TimeMasking()
    (1): FrequencyMasking()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(28, 28, kernel_size=(7,), stride=(1,), padding=(3,))
      (2): GELU(approximate='none')
      (3): PermuteBlock()
    )
    (BLSTMs): LSTM(28, 256, num_layers=3, batch_first=True, dropout=0.25, bidirectional=True)
    (pBLSTMs): Sequential(
      (0): LockedDropout()
      (1): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (2): LockedDropout()
      (3): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
    )
    (dropout_layer): LockedDropout()
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=512, out_features=512, bias=True)
      (4): GELU(a

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.TimeMasking_0,-,"[64, 1683, 28]",NaN,NaN
1_augmentations.FrequencyMasking_1,-,"[64, 1683, 28]",NaN,NaN
2_encoder.embedding.PermuteBlock_0,-,"[64, 28, 1683]",NaN,NaN
3_encoder.embedding.Conv1d_1,"[28, 28, 7]","[64, 28, 1683]",5516.0,9236304.0
4_encoder.embedding.GELU_2,-,"[64, 28, 1683]",NaN,NaN
5_encoder.embedding.PermuteBlock_3,-,"[64, 1683, 28]",NaN,NaN
6_encoder.LSTM_BLSTMs,-,"[82600, 512]",3739648.0,3727360.0
7_encoder.pBLSTMs.LockedDropout_0,-,"[82600, 512]",NaN,NaN
8_encoder.pBLSTMs.1.LSTM_blstm,-,"[41252, 512]",2625536.0,2621440.0


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

Initializing loss function criterion using  CTCLOss, and for this task I used AdamW as the optimizer where i tuned paramater like learning rate, and weight decay. For scheduler I used CosineAnnealingLR, ReduceLROnPlateau, and stepLR. Decoder I implemented CTCBeamDecoder to enhance the preduction accuracy.



In [ ]:
#TODO
criterion = nn.CTCLoss(blank= CMUdict.index('')) # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr']) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=LABELS,
                         beam_width=3,
                         num_processes= 1,
                         log_probs_input=True) #TODO

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
#                                                        mode= 'min',
#                                                        factor= 0.5,
#                                                        patience= 3,
#                                                        threshold= 1e-3,
#                                                        verbose=True)#TODO

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])



# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

Decode prediction The function that decodes the output predictions from the model using a CTCBeamDecoder.

Calculate levenshtein method, the function for calculating the average Levenshtein distance, which measures the edit distance between predicted phoneme sequences and the ground truth labels across a batch. Which is used to evaluate the model's performance by quantifying how many insertions, deletions, or substitutions are required to convert the predicted sequences into the actual phoneme sequences.




In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    output = torch.transpose(output, 0, 1)
    output, _, _, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        logits=output[i][0][:out_seq_len[i][0]]
        try:
            pred_strings.append(''.join([PHONEME_MAP[i] for i in logits]))
        except:
            print(logits)
            
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]   # TODO
        label_string = ''.join(PHONEME_MAP[l] for l in label[i][:label_lens[i]]) #TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

# Test Implementation

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 811, 41])
torch.Size([811, 64, 41]) torch.Size([64, 360])
tensor(7.1916, device='cuda:0', grad_fn=<MeanBackward0>)
229.65625


# WandB

You will need to fetch your api key from wandb.ai

In [33]:
import wandb
wandb.login(key="65ab01e861811be49c0d4f76312a0729eb26cdfc")


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Here is the link to the wandb to visualize the runs that I did unfortunetly my trial for wandb has expired and force me to downgrade to 5GB storage and I tried to delete some runs to free up space. Here is the link to see my run Url: https://wandb.ai/angeizabayo05-carnegie-mellon-university, run_id that gave me the high cutoff **fqxhsk3w**

In [34]:
run = wandb.init(
    name = "New_run_submission-08-11-2022_50Epoch", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: angeizabayo05 (angeizabayo05-carnegie-mellon-university). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241108_201103-fqxhsk3w
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run New_run_submission-08-11-2022_50Epoch
wandb: ⭐️ View project at https://wandb.ai/angeizabayo05-carnegie-mellon-university/hw3p2-ablations
wandb: 🚀 View run at https://wandb.ai/angeizabayo05-carnegie-mellon-university/hw3p2-ablations/runs/fqxhsk3w


# Train Functions

In [35]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
 
    batch_bar.close() #close the progress bar
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss,val_dist

## Training Setup

In [36]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [37]:
epoch_model_path = '/kaggle/working/epoch_model.pth'#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = '/kaggle/working/best_model.pth'#TODO set best model path# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.

# Set paths for saving models
checkpoint_dir = "./checkpoints"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Define the model paths
epoch_model_path = '/kaggle/working/epoch_model.pth'#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = '/kaggle/working/best_model.pth'#TODO set best model path

For training loop, I change the above hyperparameter in config dictionary and Scheduler to get the required validation where i started with ReduceLROnPlateau which gave me low cutoff and not working and change to CosineAnnealingLR and tried StepLR as well but it was not perform well as well.

I changed the learning rate from given 2e-3 to 1e-3
Used Batch size of 64, 32
for scheduler used the T-max equal to the number of epoch
for encoder I used dropout and decoder function of 0.25


for this work training loop  save and display the training loss, validation loss, and validation distance, where I saved the checkpoint and best model in wandb


In [38]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer)#TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)#TODO
#     scheduler.step(valid_dist)

    scheduler.step()

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


	Train Loss 2.2528	 Learning Rate 0.0010000
	Val Dist 24.7807%	 Val Loss 1.0999
Saved epoch model
Saved best model

Epoch: 2/50


	Train Loss 0.9800	 Learning Rate 0.0009990
	Val Dist 14.1904%	 Val Loss 0.6371
Saved epoch model
Saved best model

Epoch: 3/50


	Train Loss 0.7444	 Learning Rate 0.0009961
	Val Dist 12.1007%	 Val Loss 0.5392
Saved epoch model
Saved best model

Epoch: 4/50


...

	Train Loss 0.1147	 Learning Rate 0.0000157
	Val Dist 4.5161%	 Val Loss 0.2541
Saved epoch model
Saved best model

Epoch: 48/50


	Train Loss 0.1085	 Learning Rate 0.0000089
	Val Dist 4.4684%	 Val Loss 0.2525
Saved epoch model
Saved best model

Epoch: 49/50


	Train Loss 0.1101	 Learning Rate 0.0000039
	Val Dist 4.5405%	 Val Loss 0.2493
Saved epoch model

Epoch: 50/50


	Train Loss 0.1036	 Learning Rate 0.0000010
	Val Dist 4.5023%	 Val Loss 0.2550
Saved epoch model


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         lr ████████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
wandb: train_loss █▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: valid_dist █▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: valid_loss █▄▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:         lr 0.0
wandb: train_loss 0.10363
wandb: valid_dist 4.50225
wandb: valid_loss 0.25504
wandb: 
wandb: 🚀 View run New_run_submission-08-11-2022_50Epoch at: https://wandb.ai/angeizabayo05-carnegie-mellon-university/hw3p2-ablations/runs/fqxhsk3w
wandb: ⭐️ View project at: https://wandb.ai/angeizabayo05-carnegie-mellon-university/hw3p2-ablations
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 2 other file(s)
wandb: Find logs at: ./wandb/run-20241108_201103-fqxhsk3w/logs


# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder



TEST_BEAM_WIDTH = 20 #TODO

test_decoder    = CTCBeamDecoder(labels=LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True)#TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)
        h = torch.permute(h, (1, 0, 2))
    prediction_string= decode_prediction(h, lh, test_decoder, LABELS)# TODO call decode_prediction
    #TODO save the output in results array.
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 41/41 [00:35<00:00,  1.14it/s]


Generate the csv file of prediction

In [40]:
data_dir = "/kaggle/input/random-sub/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results

df.to_csv('submission_8_50Epoch.csv', index = False)

In [41]:
!kaggle competitions submit -c 11-785-hw3p2-f24 -f submission_8_50Epoch.csv -m "I made it!"


100%|████████████████████████████████████████| 209k/209k [00:00<00:00, 1.16MB/s]
Successfully submitted to Automatic Speech Recognition (ASR)